In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [3]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

In [4]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_23040s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [5]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

In [6]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg.shape

(1258567, 20)

In [7]:
tabs_assLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,30,1,1,11088.0,-346.181835,-79.056680,0,2,1631.000000,3717.548753,0.000000,0.497617,1,-346.181835,-1147,205.066526,-842.203459,31.628339,-65.838388,0.508633
1,64,1,1,11128.0,-453.878455,-85.030449,0,2,909.000000,3401.481115,0.000000,0.499643,0,-453.878455,None,569.364700,-683.257407,62.820835,-47.924593,0.508633
2,121,1,1,11258.0,-507.020338,-90.344656,0,3,701.728907,3031.372645,0.314326,0.341892,1,-507.020338,-1704,674.119700,-496.740654,58.122542,-33.829709,0.762949
3,163,1,1,11335.0,-326.899810,-81.912587,0,3,1047.351458,2927.671370,0.337591,0.332139,0,-326.899810,None,499.887144,-444.230796,37.719469,-31.341331,0.762949
4,194,1,1,11388.0,-310.766187,-95.505788,0,3,2927.100192,2855.000000,0.585318,0.000000,1,-310.766187,"-2034,-2028,-195",-448.020769,-406.421344,-32.377834,-29.039490,0.762949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609405,1325651,93,4,1326030.0,133.539612,38.116777,0,2,204.498871,741.501129,0.499999,0.499999,1,133.539612,1312029,-624.600898,465.393465,-48.135517,28.070530,8.158247
609406,1325652,93,4,1326031.0,142.838259,45.492560,0,3,549.676033,745.369356,0.330917,0.341271,1,142.838259,1325483,72.547889,473.206061,4.949636,27.940322,12.237370
609407,1325653,93,4,1286482.0,90.385433,39.499436,0,2,578.536093,775.000000,0.498696,0.000015,1,90.385433,1285962,130.836090,533.050599,9.352547,32.213243,8.158247
609408,1325658,93,4,1286496.0,203.680267,41.058705,0,3,627.372735,888.663548,0.341909,0.334105,1,203.680267,1273112,229.470675,762.615047,22.893882,51.636292,12.237370


In [13]:
idx=11088
tabs_assLrg.loc[tabs_assLrg['assoc_larger']==idx]

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,30,1,1,11088.0,-346.181835,-79.056680,0,2,1631.000000,3717.548753,0.000000,0.497617,1,-346.181835,-1147,205.066526,-842.203459,31.628339,-65.838388,0.508633
48,1147,1,1,11088.0,-11863.373024,-358.745243,20,31,1628.160959,3713.114931,0.291368,0.245562,1,-11863.373024,-30,206.500988,-839.969247,31.491211,-65.557728,7.883811


In [14]:
tabs.loc[tabs['id']==idx]

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
11087,11088,1,2,16364.0,-1464.595824,-96.558989,5,8,816.776495,1858.285593,0.528658,0.170725,1,-1464.595824,-10797,-212.697352,843.247868,-24.734170,57.851777,8.158069
30919,11088,2,2,NaN,-674.337952,-99.708132,1,4,838.965595,1857.264547,0.358560,0.254665,0,790.257872,None,-190.354646,842.241586,-21.891865,57.687973,4.079115


In [6]:
tabs_NoassLrg.head(20)

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-427.401395,-83.874285,0,2,1994.550491,3813.550491,0.497444,0.497444,0,-427.401395,None,21.669561,-890.392278,4.546983,-73.370069,0.508633
1,2,1,1,NaN,-635.447272,-82.187646,0,3,2046.357913,3811.291415,0.338977,0.321320,1,-635.447272,"-1097,-1095",-4.455403,-889.221020,-0.923948,-73.185057,0.762949
2,3,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1095,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
3,4,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1099,105.836138,-888.242242,21.967109,-72.768028,0.762949
4,5,1,1,NaN,-412.025671,-77.163422,0,2,1877.470658,3809.470658,0.499138,0.499138,1,-412.025671,"-1098,9854",80.715163,-888.407168,16.705742,-72.902785,0.508633
5,6,1,1,NaN,-935.759582,-95.803365,1,4,1882.287636,3807.993468,0.261344,0.392628,1,-935.759582,"-1103,-1102,-1101,-1098,9854",78.286886,-887.659248,16.083357,-72.790183,1.017266
6,7,1,1,NaN,-443.545833,-95.669428,1,2,1884.596392,3804.000000,0.490621,0.000000,1,-443.545833,"-1103,-1101,-1100",77.125053,-885.643926,15.550300,-72.469990,0.508633
7,8,1,1,NaN,-1313.586161,-88.136336,0,6,1878.480681,3804.200837,0.223440,0.468238,1,-1313.586161,"-1103,-1102,-1098",80.209067,-885.748986,16.194591,-72.476152,1.525899
8,9,1,1,NaN,-419.753313,-87.217274,0,2,2269.549158,3782.549158,0.497578,0.497578,1,-419.753313,-10,-116.992457,-874.588463,-21.659940,-70.619873,0.508633
9,10,1,1,NaN,-657.718144,-91.582987,0,3,2267.644678,3781.366446,0.338429,0.340708,1,-657.718144,"-1111,-9",-116.031302,-873.993202,-21.380347,-70.536691,0.762949


In [7]:
# Procesing objects

# Assembling pandas dataframe with columns
Feature = pd.DataFrame({'id':np.unique(tabs_NoassLrg['id'])
             })
Feature

,id
0,1
1,2
2,3
3,4
4,5
...,...
978341,1351962
978342,1351963
978343,1351964
978344,1351965


In [8]:
for i in np.arange(0,Feature.shape[0]):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == Feature['id'][i]].copy().reset_index(drop=True)
    Feature.loc[i,'Life'] = np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1
    
    maxSzin = (snglFtr['size_s'].values==np.max(snglFtr['size_s'].values)).nonzero()[0][0]
    
    Feature.loc[i,'scale'] = snglFtr.loc[maxSzin, 'scale']
    Feature.loc[i,'Flux'] = snglFtr.loc[maxSzin, 'flux']
    Feature.loc[i,'Lat'] = snglFtr.loc[maxSzin, 'lat']
    Feature.loc[i,'Lon'] = snglFtr.loc[maxSzin, 'lon']
    Feature.loc[i,'x'] = snglFtr.loc[maxSzin, 'x']
    Feature.loc[i,'xerr'] = snglFtr.loc[maxSzin, 'xerr']
    Feature.loc[i,'y'] = snglFtr.loc[maxSzin, 'y']
    Feature.loc[i,'yerr'] = snglFtr.loc[maxSzin, 'yerr']
    Feature.loc[i,'MaxSize'] = snglFtr.loc[maxSzin, 'size']
    Feature.loc[i,'xs'] = snglFtr.loc[maxSzin, 'xs']
    Feature.loc[i,'ys'] = snglFtr.loc[maxSzin, 'ys']
    Feature.loc[i,'size_s'] = snglFtr.loc[maxSzin, 'size_s']
    Feature.loc[i,'maxB'] = snglFtr.loc[maxSzin, 'maxB']
    
Feature.head()

,id,Life,scale,Flux,Lat,Lon,x,xerr,y,yerr,MaxSize,xs,ys,size_s,maxB
0,1,1.0,1.0,-427.401395,-73.370069,4.546983,1994.550491,0.497444,3813.550491,0.497444,2.0,21.669561,-890.392278,0.508633,-83.874285
1,2,1.0,1.0,-635.447272,-73.185057,-0.923948,2046.357913,0.338977,3811.291415,0.321320,3.0,-4.455403,-889.221020,0.762949,-82.187646
2,3,1.0,1.0,-401.259592,-73.077809,-0.515724,2042.486745,0.499824,3810.000000,0.000086,2.0,-2.502384,-888.572155,0.508633,-74.707427
3,4,1.0,1.0,-744.679039,-72.768028,21.967109,1827.657345,0.335591,3809.082584,0.570318,3.0,105.836138,-888.242242,0.762949,-95.060790
4,5,1.0,1.0,-412.025671,-72.902785,16.705742,1877.470658,0.499138,3809.470658,0.499138,2.0,80.715163,-888.407168,0.508633,-77.163422


In [9]:
Feature.to_pickle('data\SWAMIS\\100_frames\swamis_23040s_RA.pnds')

In [10]:
np.nanmin(Feature['Life'])

1.0